In [1]:
from torchvision.io.image import read_image
from torchvision.transforms.functional import normalize, resize, to_pil_image
from torchcam.methods import SmoothGradCAMpp
import numpy as np
import torch
import matplotlib.pyplot as plt
from torchcam.utils import overlay_mask
import numpy as np
import pandas as pd
import torchvision
import os
import shutil
from d2l import torch as d2l
from collections import Counter

In [2]:
data_dir = r'C:\AI_project\data\ImageNet'
def read_csv_labels(fname):
    """读取 fname 来给标签字典返回一个文件名"""
    with open(fname, 'r') as f:
        lines = f.readlines()
    tokens = [l.rstrip().split(' ')[1] for l in lines]
    return tokens

def read_csv_labels_1(fname):
    """读取 fname 来给标签字典返回一个文件名"""
    with open(fname, 'r') as f:
        lines = f.readlines()
    tokens = [l.rstrip() for l in lines]
    return tokens

labels_2 = read_csv_labels(os.path.join(data_dir,'val.txt'))
labels_1 = read_csv_labels_1(os.path.join(data_dir,'result.txt'))
labels = read_csv_labels_1(os.path.join(data_dir,'ILSVRC2012_validation_ground_truth.txt'))
files = sorted(list(set(labels)))
np_labels = np.array(labels_2, dtype=int)
np_labels_1 = np.array(labels, dtype=int)
result_1 = np_labels - np_labels_1 
(result_1==0).sum()

50

In [3]:
filename =os.listdir(os.path.join(data_dir, 'validation'))

In [4]:
def copyfile(filename, target_dir):
    """将文件复制到目标目录"""
    os.makedirs(target_dir, exist_ok=True)
    shutil.copy(filename, target_dir)

def reorg_valid(data_dir, labels):
    for i in range(len(labels)):
        label = labels[i]
        file = filename[i]
        fname = os.path.join(data_dir, 'validation', file)
        copyfile(fname, os.path.join(data_dir, 'valid',label))

In [5]:
reorg_valid(data_dir=data_dir, labels= labels_2)

In [6]:
transform_test = torchvision.transforms.Compose([
    torchvision.transforms.Resize([256,256]),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.4914, 0.4822, 0.4465],
                                     [0.2023, 0.1994, 0.2010])])

In [7]:
valid_ds = torchvision.datasets.ImageFolder(
    os.path.join(data_dir, 'valid'), transform=transform_test)

valid_iter = torch.utils.data.DataLoader(valid_ds, batch_size=10, shuffle=False, drop_last=True,num_workers=0)

In [21]:
import torchvision.models as tm

devices = d2l.try_all_gpus()
net_1 = tm.alexnet(pretrained=True).eval()
preds = []
test_label = []
i = 0
right = 0
wrong = 0
for X, y in valid_iter:
    y_hat = net_1(X)
    preds.extend(y_hat.argmax(dim=1).type(torch.int32).cpu().numpy())
    test_label.extend(y.numpy())
    '''if y_hat.argmax(dim=1).type(torch.int32).cpu().numpy() == y.numpy():
        right += 1
    else:
        wrong += 1'''

    i += 1
    if i == 40:
        break

In [23]:
np_preds = np.array(preds)
np_testlabel = np.array(test_label)
result = np_preds - np_testlabel
(result == 0).sum() / len(preds)

0.1925